# Text Classification with OCI Generative AI

### What this file does:
Demonstrates text classification using OCI Generative AI Cohere models. Shows how to prompt the model to classify customer questions into predefined categories using a custom preamble.

**Documentation to reference:**
- OCI Gen AI: https://docs.oracle.com/en-us/iaas/Content/generative-ai/pretrained-models.htm
- Cohere Command Models: https://docs.cohere.com/docs/command-r
- OCI Python SDK: https://github.com/oracle/oci-python-sdk/tree/master/src/oci/generative_ai_inference

**Relevant slack channels:**
- #generative-ai-users: *for questions on OCI Gen AI*
- #igiu-innovation-lab: *general discussions on your project*
- #igiu-ai-learning: *help with sandbox environment or help with running this code*

**Env setup:**
- sandbox.yaml: Contains OCI config, compartment, and other details.
- .env: Load environment variables (e.g., API keys if needed).
- configure cwd for jupyter match your workspace python code: 
    -  vscode menu -> Settings > Extensions > Jupyter > Notebook File Root
    -  change from `${fileDirname}` to `${workspaceFolder}`


**How to run in notebook:**
- Make sure your runtime environment has all dependencies and access to required config files.
- Run the notebook cells in order.

### Supported models (https://docs.oracle.com/en-us/iaas/Content/generative-ai/chat-models.htm)
- cohere.command-a-03-2025
- cohere.command-r-08-2024
- cohere.command-r-plus-08-2024


### Step 1: Set up variables and imports

In [ ]:
# set up the  variables


from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import *
import oci
import json, os 
from dotenv import load_dotenv
from envyaml import EnvYAML

#####
#make sure your sandbox.yaml file is setup for your environment. You might have to specify the full path depending on  your `cwd` 
# you can also try making your cwd ofr jupyter match your workspace python code: 
# vscopde menu -> Settings > Extensions > Jupyter > Notebook File Root
# change from ${fileDirname} to ${workspaceFolder}
#####

SANDBOX_CONFIG_FILE = "sandbox.yaml"
load_dotenv()

LLM_MODEL = "cohere.command-r-plus-08-2024" 
PREAMBLE = """
        answer in three bullets. respond in hindi 
"""
MESSAGE = """
        "why is indian cricket team so good"
"""

llm_service_endpoint= "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

llm_client = None
llm_payload = None

### Step 2: Load configuration and OCI config

In [ ]:

scfg = EnvYAML(SANDBOX_CONFIG_FILE)
if scfg is None or "oci" not in scfg or "bucket" not in scfg:
    raise RuntimeError("Invalid sandbox configuration.")

#read the oci config
config = oci.config.from_file(os.path.expanduser(scfg["oci"]["configFile"]),scfg["oci"]["profile"])

### Step 3: Set up chat request with classification preamble

In [ ]:


cohere_chat_request = CohereChatRequest()
cohere_chat_request.preamble_override = """
        
        You are a call classifier you carefully analyze teh question and classify it into one of the following categories. 
        you then return just the category in response
        
        categories: billing, outage, program, service
        
        eg: 
        question:  can i pay my bill by creditcard
        answer: billing
        question: i need to cancel my service as i am moving
        answer: service
        question: when will by power come back on?
        answer: outage
        
        """
cohere_chat_request.is_stream = False 
cohere_chat_request.max_tokens = 500
cohere_chat_request.temperature = 0.75
cohere_chat_request.top_p = 0.7
cohere_chat_request.frequency_penalty = 1.0


### Step 4: Set up chat details

In [ ]:

chat_detail = ChatDetails()
chat_detail.serving_mode = OnDemandServingMode(model_id=LLM_MODEL)
chat_detail.compartment_id = scfg["oci"]["compartment"]
chat_detail.chat_request = cohere_chat_request

### Step 5: Initialize LLM client

In [ ]:

# set up the LLM client 
llm_client = GenerativeAiInferenceClient(
                config=config,
                service_endpoint=llm_service_endpoint,
                retry_strategy=oci.retry.NoneRetryStrategy(),
                timeout=(10,240))

### Step 6: Make classification request

In [ ]:
cohere_chat_request.message = "why is my bill so high"

llm_response = llm_client.chat(chat_detail)
llm_text = llm_response.data.chat_response.text

### Step 7: Print classification result

In [ ]:
print("**************************Chat Result**************************")
#llm_text = llm_response.data.chat_response.text
print(llm_response.data.chat_response.text)

## Exercise 


Merge your RAG Agent & Single step tool into same code and use clasiification to decide which to invoke
eg: 
1. If the queston is about weather, cll the single step tool
2. if the call is about batteries use the RAG to answer
3. if its about something else say you can only answer questions about weather or batteries

